In [ ]:
!pip install tensorflow-gpu==2.3.0

     |████████████████████████████████| 320.4 MB 46 kB/s 
     |████████████████████████████████| 459 kB 8.7 MB/s 
     |████████████████████████████████| 2.9 MB 45.5 MB/s 
     |████████████████████████████████| 20.1 MB 1.3 MB/s 
  Attempting uninstall: numpy
    Found existing installation: numpy 1.19.5
    Uninstalling numpy-1.19.5:
      Successfully uninstalled numpy-1.19.5
  Attempting uninstall: tensorflow-estimator
    Found existing installation: tensorflow-estimator 2.6.0
    Uninstalling tensorflow-estimator-2.6.0:
      Successfully uninstalled tensorflow-estimator-2.6.0
  Attempting uninstall: h5py
    Found existing installation: h5py 3.1.0
    Uninstalling h5py-3.1.0:
      Successfully uninstalled h5py-3.1.0
  Attempting uninstall: gast
    Found existing installation: gast 0.4.0
    Uninstalling gast-0.4.0:
      Successfully uninstalled gast-0.4.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour 

In [ ]:
#查看GPU狀態
!nvidia-smi

In [ ]:
!nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2020 NVIDIA Corporation
Built on Wed_Jul_22_19:09:09_PDT_2020
Cuda compilation tools, release 11.0, V11.0.221
Build cuda_11.0_bu.TC445_37.28845127_0


In [ ]:
#連接google雲端
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
%cp /content/drive/MyDrive/AVMap/SIG_GAT/dataset_classify.zip /content/dataset.zip
!unzip "dataset.zip"

Archive:  dataset.zip
   creating: dataset/train/
   creating: dataset/train/line/
  inflating: dataset/train/line/XN1A30S700_XN1A30S7BC_1.jpg  
  inflating: dataset/train/line/XN1A30S700_XN1A30S7BC_2.jpg  
  inflating: dataset/train/line/XN1A30S700_XN1A30S7PK_1.jpg  
  inflating: dataset/train/line/XN1A30S700_XN1A30S7PK_2.jpg  
  inflating: dataset/train/line/XN1A30S700_XN1A30S7VA_1.jpg  
  inflating: dataset/train/line/XN1A30S700_XN1A30S7VA_3.jpg  
  inflating: dataset/train/line/XN1A30S800_XN1A30S82M_1.jpg  
  inflating: dataset/train/line/XN1A30S800_XN1A30S82M_2.jpg  
  inflating: dataset/train/line/XN1A30S800_XN1A30S8GK_1.jpg  
  inflating: dataset/train/line/XN1A30S800_XN1A30S8GK_2.jpg  
  inflating: dataset/train/line/XN1A30S800_XN1A30S8ME_1.jpg  
  inflating: dataset/train/line/XN1A30S800_XN1A30S8ME_2.jpg  
  inflating: dataset/train/line/XN1A30S800_XN1A30S8SJ_1.jpg  
  inflating: dataset/train/line/XN1A30S800_XN1A30S8SJ_2.jpg  
  inflating: dataset/train/line/XN1A30S800_XN1A30

In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers 
from tensorflow.keras.layers import Dense
from tensorflow.keras.applications import EfficientNetB7, EfficientNetB7, EfficientNetB1, EfficientNetB0
from tensorflow.keras.applications.efficientnet import preprocess_input, decode_predictions
from tensorflow.keras.preprocessing import image
from tensorflow.python.keras.engine import training
from tensorflow.python.keras.applications import imagenet_utils
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import Model

In [ ]:
tf.__version__

'2.3.0'

In [ ]:
#資料前處理
#zoom_range 聚焦 rotation_range 旋轉 horizontal_flip 水平翻轉 vertical_flip 垂直翻轉

train_datagen = tf.keras.preprocessing.image.ImageDataGenerator(
    preprocessing_function=preprocess_input, 
    horizontal_flip=True,
    vertical_flip=True,
    zoom_range=0.,
    rotation_range=0.,
    width_shift_range=0., 
    height_shift_range=0.)


test_datagen = tf.keras.preprocessing.image.ImageDataGenerator(preprocessing_function=preprocess_input)

In [ ]:
#target_size 和 batch_size 可調整
train_generator = train_datagen.flow_from_directory(
        'dataset/train',
        target_size=(320, 320),
        batch_size=16,
        class_mode='categorical')

validation_generator = test_datagen.flow_from_directory(
        'dataset/val',
        target_size=(320, 320),
        batch_size=16,
        class_mode='categorical')

Found 451 images belonging to 4 classes.
Found 47 images belonging to 4 classes.


In [ ]:
epochs = 200

early = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=1, verbose=1, mode='auto')
filepath = "weights-{epoch:02d}-{val_accuracy:.2f}.hdf5"
checkpoint = tf.keras.callbacks.ModelCheckpoint(filepath, monitor='val_accuracy', verbose=1, save_best_only=False, mode='max')
csvlog = tf.keras.callbacks.CSVLogger('log.csv')
callbacks = [csvlog, checkpoint]

In [ ]:
#讀取網路架構，可選擇其他的網路架構，請參考 keras API:https://www.tensorflow.org/api_docs/python/tf/keras/applications
#第一次訓練時執行此行
model = EfficientNetB0(include_top = False, weights='imagenet', pooling = 'avg', classes = 4)

x = model.output
#Dropout層可加可不加，如果overfitting就加
x = layers.Dropout(0.2, name='top_dropout')(x)
output = Dense(units = 4, activation="softmax")(x)

model = Model(inputs=model.input, outputs = output, name='efficientnet')

model.summary()

Model: "efficientnet"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, None, None,  0                                            
__________________________________________________________________________________________________
rescaling (Rescaling)           (None, None, None, 3 0           input_1[0][0]                    
__________________________________________________________________________________________________
normalization (Normalization)   (None, None, None, 3 7           rescaling[0][0]                  
__________________________________________________________________________________________________
stem_conv_pad (ZeroPadding2D)   (None, None, None, 3 0           normalization[0][0]              
_______________________________________________________________________________________

In [ ]:
#learning_rate 可調整
adam = tf.keras.optimizers.Adam(learning_rate=0.001)

In [ ]:
#開始訓練
#model.compile(optimizer = SGD_AGC(lr=1e-3), loss = 'categorical_crossentropy', metrics = ['accuracy'])
model.compile(optimizer = adam, loss = 'categorical_crossentropy', metrics = ['accuracy'])

history = model.fit_generator(
    train_generator,
    steps_per_epoch=len(train_generator),
    epochs=epochs,
    validation_data=validation_generator,
    validation_steps=len(validation_generator),
    verbose=1,
    use_multiprocessing=True,
    workers=4,
    callbacks=callbacks
)


Instructions for updating:
Please use Model.fit, which supports generators.
Epoch 1/200
29/29 [==============================] - ETA: 0s - loss: 0.3648 - accuracy: 0.8914WARNING:tensorflow:multiprocessing can interact badly with TensorFlow, causing nondeterministic deadlocks. For high performance data pipelines tf.data is recommended.

Epoch 00001: saving model to weights-01-0.68.hdf5
29/29 [==============================] - 26s 895ms/step - loss: 0.3648 - accuracy: 0.8914 - val_loss: 1.5693 - val_accuracy: 0.6809
Epoch 2/200
29/29 [==============================] - ETA: 0s - loss: 0.2098 - accuracy: 0.9401WARNING:tensorflow:multiprocessing can interact badly with TensorFlow, causing nondeterministic deadlocks. For high performance data pipelines tf.data is recommended.

Epoch 00002: saving model to weights-02-0.68.hdf5
29/29 [==============================] - 23s 781ms/step - loss: 0.2098 - accuracy: 0.9401 - val_loss: 2.1480 - val_accuracy: 0.6809
Epoch 3/200
29/29 [=================

Process Keras_worker_ForkPoolWorker-1139:
Process Keras_worker_ForkPoolWorker-1137:
Process Keras_worker_ForkPoolWorker-1140:
Traceback (most recent call last):
Process Keras_worker_ForkPoolWorker-1138:
  File "/usr/lib/python3.7/multiprocessing/process.py", line 297, in _bootstrap
    self.run()
Traceback (most recent call last):
Traceback (most recent call last):
  File "/usr/lib/python3.7/multiprocessing/process.py", line 99, in run
    self._target(*self._args, **self._kwargs)
Traceback (most recent call last):
  File "/usr/lib/python3.7/multiprocessing/pool.py", line 110, in worker
    task = get()
  File "/usr/lib/python3.7/multiprocessing/process.py", line 297, in _bootstrap
    self.run()
  File "/usr/lib/python3.7/multiprocessing/process.py", line 297, in _bootstrap
    self.run()
  File "/usr/lib/python3.7/multiprocessing/process.py", line 297, in _bootstrap
    self.run()
  File "/usr/lib/python3.7/multiprocessing/queues.py", line 352, in get
    res = self._reader.recv_byte

KeyboardInterrupt: ignored

In [ ]:
#下載model
from google.colab import files
#files.download("backup/custom-yolov4-detector_best.weights")
files.download("/content/drive/My Drive/雲科大產學合作/207442_365770_20201125_友達玻璃瑕疵會議.pptx")

In [ ]:
%cd /content/drive/MyDrive/AVMap/SIG_GAT/
%cp /content/dataset.zip /content/drive/MyDrive/AVMap/SIG_GAT/dataset_classify.zip

/content/drive/MyDrive/AVMap/SIG_GAT


In [ ]:
from google.colab import files
files.upload()

In [ ]:
!unzip "dataset.zip"

In [ ]:
import tensorflow as tf
from tensorflow import keras 
from tensorflow.python.framework.convert_to_constants import convert_variables_to_constants_v2
import numpy as np

In [ ]:
tf.executing_eagerly()

True

In [ ]:
frozen_out_path = "/content/"
frozen_graph_filename = "weights-95-100"

In [ ]:
weights_path = 'weights-95-100.hdf5'
model = tf.keras.models.load_model(weights_path)
model.summary()

In [ ]:
full_model = tf.function(lambda x: model(x))
full_model = full_model.get_concrete_function(tf.TensorSpec(model.inputs[0].shape, model.inputs[0].dtype))

In [ ]:
frozen_func = convert_variables_to_constants_v2(full_model)
frozen_func.graph.as_graph_def()

In [ ]:
layers = [op.name for op in frozen_func.graph.get_operations()]

In [ ]:
print("-" * 60)
print("Frozen model layers: ")
for layer in layers:
  print(layer)
print("-" * 60)
print("Frozen model inputs: ")
print(frozen_func.inputs)
print("Frozen model outputs: ")
print(frozen_func.outputs)

In [ ]:
tf.io.write_graph(graph_or_graph_def=frozen_func.graph, logdir=frozen_out_path, name=f"{frozen_graph_filename}.pb", as_text=False)

'/content/weights-95-100.pb'

In [ ]:
tf.io.write_graph(graph_or_graph_def=frozen_func.graph, logdir=frozen_out_path, name=f"{frozen_graph_filename}.pbtxt", as_text=True)

'/content/weights-95-100.pbtxt'

In [ ]:
https://blog.csdn.net/caroline_wendy/article/details/109776590

In [ ]:
import cv2

In [ ]:
model_path = 'weights-95-100.pb'
config_path = 'weights-95-100.pbtxt'
net = cv2.dnn.readNetFromTensorflow(model_path)